In [87]:
import numpy as np
from tabulate import tabulate
from scipy import linalg

#### Definições

In [88]:
n_to_test = [4, 6, 8, 10]
eps = 2.2 * 1e-16

$e_\text{algoritmo} = \frac{||\tilde{X}_\text{algoritmo} - A^\dagger||_2}{\epsilon_M ||A^\dagger||_2 \mathcal{k}_2(A)}$

In [89]:
# fator de estabilidade
def e_alg(X_alg, A_pinv):
    # a norma padrão do numpy é 2
    norm_diff = np.linalg.norm(X_alg - A_pinv)
    norm_A_pinv = np.linalg.norm(A_pinv)
    # igualmente, a condição padrão é 2
    cond_A = np.linalg.cond(A_pinv)
    return norm_diff / (eps * norm_A_pinv * cond_A)

$res_\text{algoritmo} = \frac{||\tilde{X}_\text{algoritmo}A - I_n||_2}{||A||_2 ||\tilde{X}_\text{algoritmo}||_2}$

In [90]:
# erro residual
def res_alg(X_alg, A):
    id = np.eye(A.shape[0]) # assumindo que A é quadrada
    norm_diff = np.linalg.norm(X_alg @ A - id)
    norm_A = np.linalg.norm(A)
    norm_X_alg = np.linalg.norm(X_alg)
    return norm_diff / (norm_A * norm_X_alg)

In [91]:
def chol(A):
    M = A.T @ A
    # find the Cholesky factorization M = R^T R
    R = np.linalg.cholesky(M, upper=True)
    # Solve the equation RT RX = AT for X
    # (by solving two equations with triangular matrices: R.T @ Y = A.T and R @ X = Y ).
    Y = np.linalg.solve(R.T, A.T)
    X = np.linalg.solve(R, Y)
    return X

In [92]:
def qr(A):
    Q, R = np.linalg.qr(A)
    return np.linalg.solve(R, Q.T)

In [95]:
def gs(a):
    # coleção de vetores q, a ser construída
    q = []
    for i in range(len(a)):
        # montando q~ (aqui chamado de qt, "q til")
        qt = a[i]
        for qi in q:
            qt = np.subtract(qt, np.dot(qi, a[i]) * qi)
        # se q~ = 0, sair
        if np.linalg.norm(qt) == 0:
            return q
        # senão, normalize
        q.append(qt / np.linalg.norm(qt))
    return q

In [ ]:
def mgs(a):
    

In [96]:
def qr_normal(A):
    Qt = gs(A.T)
    # Como sabe-se que A = QR ...
    R_inv = np.linalg.inv(Qt @ A)
    return R_inv @ Qt

In [97]:
A = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

A = np.array(A)
A

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [98]:
qr_mgs(A)

array([[ 2.11312592e+14,  2.37726667e+14,  3.69797037e+14],
       [-4.22625185e+14, -4.75453333e+14, -7.39594074e+14],
       [ 2.11312592e+14,  2.37726667e+14,  3.69797037e+14]])

In [99]:
qr(A)

array([[-5.38024702e+14,  1.07604940e+15, -5.38024702e+14],
       [ 1.07604940e+15, -2.15209881e+15,  1.07604940e+15],
       [-5.38024702e+14,  1.07604940e+15, -5.38024702e+14]])

In [100]:
def svd(A):
    U, S, Vt = np.linalg.svd(A)
    S_inv = np.diag(1 / S)
    return Vt.T @ S_inv @ U.T

#### Bateria de testes final

In [101]:
n_results = {}
for n in n_to_test:
    print('Processando n =', n)
    A = linalg.pascal(n)
    A_inv = np.linalg.inv(A) # não é necessário pseudoinversa
    cond_A = np.linalg.cond(A)
    # calculando as pseudo-inversas
    X_chol = chol(A)
    #X_qr = qr(A)
    X_qr = qr_normal(A)
    #X_qrp = qr_pivot(A)
    X_qrp = qr_mgs(A)
    #X_qrp = gs(A)
    X_svd = svd(A)
    # calculando a estabilidade
    e_chol = e_alg(X_chol, A_inv)
    e_qr = e_alg(X_qr, A_inv)
    e_qrp = e_alg(X_qrp, A_inv)
    e_svd = e_alg(X_svd, A_inv)
    # calculando o erro residual
    res_chol = res_alg(X_chol, A)
    res_qr = res_alg(X_qr, A)
    res_qrp = res_alg(X_qrp, A)
    res_svd = res_alg(X_svd, A)
    # salvando os resultados
    n_results[str(n)] = {
        "cond": cond_A,
        "chol": {
            "e": e_chol,
            "res": res_chol
        },
        "qr": {
            "e": e_qr,
            "res": res_qr
        },
        "qrp": {
            "e": e_qrp,
            "res": res_qrp
        },
        "svd": {
            "e": e_svd,
            "res": res_svd
        }
    }

Processando n = 4
Processando n = 6
Processando n = 8
Processando n = 10


In [102]:
table_e = []
table_res = []
for n in n_results:
    table_e.append([n, n_results[n]["cond"],
                  n_results[n]["chol"]["e"],
                  n_results[n]["qr"]["e"],
                  n_results[n]["qrp"]["e"],
                  n_results[n]["svd"]["e"]])
    
    table_res.append([n, n_results[n]["cond"],
                    n_results[n]["chol"]["res"],
                    n_results[n]["qr"]["res"],
                    n_results[n]["qrp"]["res"],
                    n_results[n]["svd"]["res"]])

In [103]:
tabulate(table_e, headers=["n", "cond(A)", "e_chol", "e_qr", "e_qrp", "e_svd"], tablefmt="html")

n,cond(A),e_chol,e_qr,e_qrp,e_svd
4,691.937,5.88844,0.0485463,0.0424779,0.0398529
6,110787,1462.75,0.00759626,0.0734328,0.024572
8,2.06452e+07,74456.2,0.000392955,0.015287,0.0103029
10,4.15521e+09,776141,0.00783289,0.00738499,0.00408852


In [104]:
tabulate(table_res, headers=["n", "cond(A)", "res_chol", "res_qr", "res_qrp", "res_svd"], tablefmt="html")

n,cond(A),res_chol,res_qr,res_qrp,res_svd
4,691.937,7.23672e-15,2.56569e-17,3.60362e-15,9.2209e-17
6,110787,1.4247e-12,1.01231e-17,1.12242e-12,1.69327e-17
8,2.06452e+07,1.00414e-10,6.76615e-18,1.83025e-11,2.32762e-17
10,4.15521e+09,3.55231e-08,2.55417e-16,4.77016e-09,1.27665e-17
